# Loading data using Xarray

We're going to try some basic operations with:

* monthly average (`Amon`) near-surface air temperatures (`tas`)
* from the latest version (`latest`) of the Met Office Hadley Centre's (`MOHC`) [Hadley Centre Global Environment Model version 3](https://www.metoffice.gov.uk/research/approach/modelling-systems/unified-model/climate-models/hadgem3) (`HadGEM3-GC31-LL`)
* running the pre-industrial control experiment (`piControl`) variant `r1i1p1f1`

We'll be:

* [Loading single or multiple NetCDF files](#Loading-single-or-multiple-NetCDF-files)
* [Selecting data for a coordinate or data variable](#Selecting-data-for-a-coordinate-or-data-variable)
* [Selecting data by value, range or condition](#Selecting-data-by-value,-range-or-condition)
* [Selecting data by position](#Selecting-data-by-position)
* [Aggregrating data by dimension](#Aggregrating-data-by-dimension)
* [Resampling time series data](#Resampling-time-series-data)
* [Grouping dimensions by value](#Grouping-dimensions-by-value)
* [Plotting line graphs](#Plotting-line-graphs)
* [Plotting colormeshes](#Plotting-colormeshes)
* [Plotting facet grids](#Plotting-facet-grids)
* Plotting on a projection - TODO
* Accessing raw data with .values - TODO

The data is available on JASMIN here:

In [ ]:
data_directory = '/badc/cmip6/data/CMIP6/CMIP/MOHC/HadGEM3-GC31-LL/piControl/r1i1p1f1/Amon/tas/gn/latest'

!ls {data_directory}

In [ ]:
!ncdump -h {data_directory}/tas_Amon_HadGEM3-GC31-LL_piControl_r1i1p1f1_gn_195001-204912.nc

First, import the libraries we will be using:

In [ ]:
from glob import glob

import matplotlib.pyplot as plt
import xarray as xr

# Set some plotting defaults
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

## Loading single or multiple NetCDF files

We can either load a single NetCDF file:

In [ ]:
dataset = xr.load_dataset(data_directory + '/tas_Amon_HadGEM3-GC31-LL_piControl_r1i1p1f1_gn_195001-204912.nc')

open multiple NetCDF files as one dataset by using wildcards:

In [ ]:
dataset = xr.open_mfdataset(data_directory + '/*.nc')

or even pass a list of NetCDF files if we want to load more than one variable.

For example, if we wanted to load near-surface air temperature (`tas`) and precipitation flux (`pr`):

<span style="color:red">*Got `ValueError: cannot guess the engine, try passing one explicitly` the original way. It could have been because `open_mfdataset` was being given a list of lists. I've flattened them to one list in my method below - JH*</span> 

In [ ]:
import itertools
paths = [glob(f'/badc/cmip6/data/CMIP6/CMIP/MOHC/HadGEM3-GC31-LL/piControl/r1i1p1f1/Amon/{variable}/gn/latest/*.nc') for variable in ['tas', 'pr']]
paths = list(itertools.chain(*paths))
dataset = xr.open_mfdataset(paths)

This will (lazily) load the data into an [`xr.DataArray` structure](https://xarray.pydata.org/en/stable/data-structures.html). Loading lazily means that the data will only be loaded into memory as and when calculcations absolutely need to be performed (for example, when plotting a graph). See [Triggering calculations manually](#Triggering-calculations-manually).

Looking at `dataset` from within the notebook, we get similar information to the `ncdump -h` command, however it will be formatted nicely with some interactive buttons to help browse the structure of the data:

In [ ]:
dataset

We can also check the coordinates and data variables using the `dims` and `data_vars` properties of the dataset, and can access individual attributes by using the `attrs` property:

In [ ]:
dataset.dims

In [ ]:
dataset.data_vars

In [ ]:
dataset.attrs['further_info_url']

## Selecting data for a coordinate or data variable

The recommended way to select data for a coordinate or data variable is to [use the dataset like a dictionary](https://xarray.pydata.org/en/stable/data-structures.html#dataset-contents):

In [ ]:
air_temperature = dataset['tas']
precipitation = dataset['pr']

### Triggering calculations manually

You will find that xarray does not actually compute the result of a calculation until it's needed (for example, in a plot).

If you want to force the data to be loaded, or a calculation to be made straight away, then you can use `.compute()`, for example:

In [ ]:
air_temperature

This is telling you about the size and shape of the result, but not its value because it has not yet been calculated.

The actual calculation will be done using [Dask](https://dask.org/) (note the "15 Tasks, 5 Chunks"):

In [ ]:
air_temperature.compute()

## Selecting data by value, range or condition

### By value

To [select a value from a dimension](https://xarray.pydata.org/en/stable/indexing.html#indexing-with-dimension-names), we can use the `.sel()` method.

For example, to select the air temperatures (for all latitudes and longitudes) for January 2000:

In [ ]:
temperature_january = air_temperature.sel(time='2000-01')
temperature_january.sizes

(Note: for simplicity we're just displaying the size of the resulting data)

When selecting by date, depending on the precision of the date we supply, more than one value may be returned.

For example, to select each of the monthly air temperatures (for all latitudes and longitudes) for 2000:

In [ ]:
temperature_2000 = air_temperature.sel(time='2000')
temperature_2000.sizes

If an exact match to your filter conditions doesn't exist, you can use the `method='nearest'` argument to [find the closest matching point](https://xarray.pydata.org/en/stable/indexing.html#nearest-neighbor-lookups). This is especially useful when selecting locations:

In [ ]:
bristol_temperature = air_temperature.sel(lat=51.4578, lon=-2.6017, method='nearest')
bristol_temperature.sizes

### By range

It's possible to use the `slice(start, end[, step])` function to specify ranges:

In [ ]:
temperature_decade = air_temperature.sel(time=slice("2000-01", "2009-12"))
temperature_decade.sizes

Bear in mind that unlike regular Python slicing, the range is inclusive of the `start` and `end` values supplied.

In [ ]:
temperature_equator = air_temperature.sel(lat=slice(-10, 10))
temperature_equator.sizes

### By condition

For more complex queries, it is often convenient to use boolean arrays, which you can give helpful names to.

For example, to select air temperatures for extreme latitudes it is winter in the northern hemisphere:

In [ ]:
is_winter = air_temperature['time'].dt.season == 'DJF'
is_extreme_latitude = abs(air_temperature['lat']) > 60

temperature_winter_poles = air_temperature.isel(time=is_winter, lat=is_extreme_latitude)
temperature_winter_poles.sizes

## Selecting data by position

Sometimes you will need to select data by position in the dataset. This can be done using the `.isel()` method, but it is generally better to use `.sel()` to select by value, unless you're selecting using a boolean array.

For example, to select the first data point in the time dimension:

In [ ]:
first_temperature = air_temperature.isel(time=0)
first_temperature.sizes

## Aggregrating data by dimension

You can [aggregate your data](https://xarray.pydata.org/en/stable/computation.html#aggregation) using the `.mean()`, `.max()`, `.min()`, `.median()`, `.std()`, `.sum()`, etc. methods.

For example, the mean temperature over all time:

In [ ]:
average_temperature = air_temperature.mean(dim='time')
average_temperature.sizes

or over every longitude as well:

In [ ]:
average_temperature = air_temperature.mean(dim=['time', 'lon'])
average_temperature.sizes

or over all the data:

In [ ]:
average_temperature = air_temperature.mean()
average_temperature.sizes

Recall that we can call `.compute()` to find out what this is:

In [ ]:
average_temperature.compute()

## Resampling time series data

When working with time series data, you [can use `.resample()`](https://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations) which has the [same syntax as Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html), in combination with the aggregation functions.

For example, to calculate the annual mean, minimum and maximum temperatures for Bristol:

In [ ]:
bristol_temperature = air_temperature.sel(lat=51.4578, lon=-2.6017, method="nearest")

annual_bristol_temperatures = (
    bristol_temperature
    .resample(time="1Y")
)

mean_annual_temperature = annual_bristol_temperatures.mean()
min_annual_temperature = annual_bristol_temperatures.min()
max_annual_temperature = annual_bristol_temperatures.max()

mean_annual_temperature.sizes

## Grouping dimensions by value

[Grouping data by value](https://xarray.pydata.org/en/stable/groupby.html) works similarly to resampling, again using a familiar Pandas-style syntax, except additional calendar functionality is available.

For example, to calcuate the mean temperature in Bristol for each season:

In [ ]:
bristol_seasonal_temperature = (
    bristol_temperature
    .groupby("time.season")
    .mean()
    .reindex(season=["DJF", "MAM", "JJA", "SON"])  # Put the values in a useful order
)

bristol_seasonal_temperature.compute()

## Plotting line graphs

TODO

Putting it all together:

In [ ]:
air_temperature = dataset['tas']

bristol_temperature = air_temperature.sel(lat=51.4578, lon=-2.6017, method="nearest")

mean_annual_temperature = bristol_temperature.resample(time="1Y").mean()
min_annual_temperature = bristol_temperature.resample(time="1Y").min()
max_annual_temperature = bristol_temperature.resample(time="1Y").max()

mean_annual_temperature.plot()

plt.fill_between(
    x=min_annual_temperature['time'].values,
    y1=min_annual_temperature.values,
    y2=max_annual_temperature.values,
    alpha=0.4,
)

## Plotting colormeshes

TODO

In [ ]:
(
    dataset['tas']
    .sel(time="2000-01")
).plot()

## Plotting facet grids

TODO

In [ ]:
(
    dataset['tas']
    .groupby('time.season')
    .mean()
    .reindex(season=['DJF', 'MAM', 'JJA', 'SON'])  # Put the values in a useful order
).plot(col='season')

## Further reading

* [xarray documentation](https://xarray.pydata.org/en/stable/index.html)
* [pandas documentation](https://pandas.pydata.org/docs/)
* [Matpltlib documentation](https://matplotlib.org/stable/contents.html)
* [NumPy documentation](https://numpy.org/doc/stable/)
* Bristol [Advanced Computing Research Centre (ACRC) training](https://www.bristol.ac.uk/acrc/acrc-training/)

## Acknowledgements

By: [James Thomas](https://github.com/jatonline/)

Last updated: 22nd April 2021